In [ ]:
pip uninstall websocket

In [ ]:
pip install websocket-client

In [ ]:
pip install nest_asyncio

In [ ]:
#skeleton 

# --- CONFIGURATION ---
CORTEX_URL = 'wss://localhost:6868'
CLIENT_ID = "Cxxgmmr3WpMiW61bnzf59flT82ZGEUGgjN8m42lM"
CLIENT_SECRET = "usD2xEZDkWEd35JYZAklttobhFs8Pst7tvDrADR6t4kaTlSCdu2HzNu8agV2VnKmW1VrjwWubpkq2vRVJvDESBGrBE3WbpzncZdEMCSI9BfWl9uQvATCaHL7mxORyQnt"
LICENSE = ""
DEBIT = 1

# Arduino Configuration
SERIAL_PORT = 'COM9'  # Change this to your Arduino port (Windows: COM3, Mac: /dev/tty.usbmodem14201, Linux: /dev/ttyACM0)
BAUD_RATE = 115200

# Command Configuration
TRAINED_COMMAND = 'lift'  # Change to your trained command
COMMAND_THRESHOLD = 0.35   # Sensitivity threshold (0.0 to 1.0)

#Commands. 


In [ ]:
import time
import json
import asyncio
import websockets
import ssl
import serial
import serial.tools.list_ports
import nest_asyncio

nest_asyncio.apply()

# --- CONFIGURATION ---
CORTEX_URL = 'wss://localhost:6868'
CLIENT_ID = "Cxxgmmr3WpMiW61bnzf59flT82ZGEUGgjN8m42lM"
CLIENT_SECRET = "usD2xEZDkWEd35JYZAklttobhFs8Pst7tvDrADR6t4kaTlSCdu2HzNu8agV2VnKmW1VrjwWubpkq2vRVJvDESBGrBE3WbpzncZdEMCSI9BfWl9uQvATCaHL7mxORyQnt"
LICENSE = ""
DEBIT = 1

# ESP32 Configuration
SERIAL_PORT = None  # Will auto-detect ESP32, or set manually like 'COM3' (Windows) or '/dev/ttyUSB0' (Linux)
BAUD_RATE = 115200  # ESP32 standard baud rate (can go higher: 230400, 460800, 921600)

# Command Configuration
TRAINED_COMMAND = 'lift'  # Change to your trained command
COMMAND_THRESHOLD = 0.35   # Sensitivity threshold (0.0 to 1.0)

# --- GLOBAL STATE ---
ws = None
ser = None
auth_token = None
session_id = None
headset_id = None
current_action = ['neutral', 0, 0]

# --- ESP32 AUTO-DETECTION ---

def find_esp32_port():
    """Auto-detect ESP32 serial port"""
    ports = serial.tools.list_ports.comports()
    
    print("\n[INFO] Scanning for ESP32...")
    for port in ports:
        print(f"  Found: {port.device} - {port.description}")
        
        # Common ESP32 identifiers
        esp32_identifiers = [
            'CP210',  # Silicon Labs CP210x
            'CH340',  # CH340 USB-Serial
            'CH9102', # CH9102 USB-Serial
            'USB-SERIAL',
            'UART',
            'ESP32',
            'Silicon Labs'
        ]
        
        description_upper = port.description.upper()
        for identifier in esp32_identifiers:
            if identifier.upper() in description_upper:
                print(f"[SUCCESS] ESP32 detected on {port.device}")
                return port.device
    
    print("[WARNING] Could not auto-detect ESP32")
    return None

# --- CORTEX API FUNCTIONS ---

async def connect_to_cortex():
    """Establishes WebSocket connection to Emotiv Cortex."""
    global ws
    try:
        ssl_context = ssl.create_default_context()
        ssl_context.check_hostname = False
        ssl_context.verify_mode = ssl.CERT_NONE
        
        ws = await websockets.connect(CORTEX_URL, ssl=ssl_context)
        print("[SUCCESS] Connected to Emotiv Cortex")
        return True
    except Exception as e:
        print(f"[ERROR] Failed to connect to Cortex: {e}")
        return False

async def send_cortex_request(method, params, request_id):
    """Sends a request to Cortex API and returns the response."""
    if not ws:
        print("[ERROR] WebSocket is not connected.")
        return None
    try:
        request = {
            "jsonrpc": "2.0",
            "method": method,
            "params": params,
            "id": request_id
        }
        await ws.send(json.dumps(request))
        response_str = await ws.recv()
        response = json.loads(response_str)
        return response
    except Exception as e:
        print(f"[ERROR] Failed to send/receive request: {e}")
        return None

async def authorize_cortex():
    global auth_token
    params = {
        "clientId": CLIENT_ID,
        "clientSecret": CLIENT_SECRET,
        "license": LICENSE,
        "debit": DEBIT
    }
    response = await send_cortex_request("authorize", params, 4)
    if response and 'result' in response:
        auth_token = response['result']['cortexToken']
        print("[SUCCESS] Authorization successful")
        return True
    print(f"[ERROR] Authorization failed")
    return False

async def query_headsets():
    global headset_id
    response = await send_cortex_request("queryHeadsets", {}, 2)
    if response and 'result' in response and len(response['result']) > 0:
        headset_id = response['result'][0]['id']
        status = response['result'][0].get('status', 'unknown')
        print(f"[SUCCESS] Headset found: {headset_id} (Status: {status})")
        return True
    print("[ERROR] No headset found")
    return False

async def control_device():
    """Connect to the headset"""
    params = {"command": "connect", "headset": headset_id}
    response = await send_cortex_request("controlDevice", params, 3)
    if response and 'result' in response:
        print("[SUCCESS] Device control established")
        return True
    return False

async def create_session():
    global session_id
    params = {
        "cortexToken": auth_token,
        "headset": headset_id,
        "status": "active"
    }
    response = await send_cortex_request("createSession", params, 5)
    
    if response:
        if 'result' in response:
            session_id = response['result']['id']
            print(f"[SUCCESS] Session created: {session_id}")
            return True
        elif 'error' in response:
            error_msg = response['error'].get('message', 'Unknown error')
            print(f"[ERROR] Session creation failed: {error_msg}")
    return False

async def subscribe_mental_commands():
    params = {
        "cortexToken": auth_token,
        "session": session_id,
        "streams": ["com"]
    }
    response = await send_cortex_request("subscribe", params, 6)
    if response and 'result' in response:
        print("[SUCCESS] Subscribed to mental commands")
        return True
    return False

async def load_profile(profile_name):
    """Load a trained profile"""
    params = {
        "cortexToken": auth_token,
        "headset": headset_id,
        "profile": profile_name,
        "status": "load"
    }
    response = await send_cortex_request("setupProfile", params, 7)
    if response and 'result' in response:
        print(f"[SUCCESS] Profile '{profile_name}' loaded")
        return True
    return False

# --- ESP32 FUNCTIONS ---

def connect_to_esp32():
    """Connects to ESP32 via serial port"""
    global ser, SERIAL_PORT
    
    # Auto-detect if not specified
    if SERIAL_PORT is None:
        SERIAL_PORT = find_esp32_port()
        if SERIAL_PORT is None:
            print("\n[INFO] Available ports:")
            ports = serial.tools.list_ports.comports()
            for port in ports:
                print(f"  {port.device}")
            print("\n[ERROR] Please set SERIAL_PORT manually in the configuration")
            return False
    
    try:
        ser = serial.Serial(
            SERIAL_PORT, 
            BAUD_RATE, 
            timeout=1,
            write_timeout=1
        )
        time.sleep(2)  # Wait for ESP32 to reset
        
        # Clear any startup messages
        ser.reset_input_buffer()
        ser.reset_output_buffer()
        
        print(f"[SUCCESS] Connected to ESP32 on {SERIAL_PORT} at {BAUD_RATE} baud")
        
        # Test communication
        ser.write(b"ping\n")
        time.sleep(0.1)
        if ser.in_waiting:
            response = ser.readline().decode('utf-8', errors='ignore').strip()
            print(f"[ESP32] Response: {response}")
        
        return True
    except Exception as e:
        print(f"[ERROR] ESP32 connection failed: {e}")
        return False

def send_to_esp32(command):
    """Sends command to ESP32"""
    if ser and ser.is_open:
        try:
            # ESP32 expects newline-terminated strings
            ser.write(f"{command}\n".encode('utf-8'))
            ser.flush()  # Ensure data is sent immediately
            return True
        except Exception as e:
            print(f"[ERROR] Failed to send to ESP32: {e}")
            return False
    return False

# --- MAIN PROCESSING ---

async def process_mental_commands():
    """Processes incoming mental command data and sends to ESP32"""
    global current_action
    
    last_command_sent = None
    command_count = 0
    
    try:
        while True:
            if not ws:
                break
                
            response = await ws.recv()
            data = json.loads(response)
            
            if 'com' in data:
                action = data['com'][0]
                power = data['com'][1]
                timestamp = data['time']
                
                current_action = [action, power, timestamp]
                
                # Determine command to send based on mental command
                if action == TRAINED_COMMAND and power > COMMAND_THRESHOLD:
                    command = "open 20"   # Command when trained action IS active
                else:
                    command = "close 20"  # Command when neutral or command NOT present
                
                # Send command to ESP32
                send_to_esp32(command)
                command_count += 1
                
                # Print status periodically to reduce console spam
                if command != last_command_sent or command_count % 20 == 0:
                    print(f"[COMMAND] Action: {action}, Power: {power:.2f}, Sent: {command}")
                    last_command_sent = command
                
            await asyncio.sleep(0.05)  # Small delay to prevent overwhelming
            
    except Exception as e:
        print(f"[ERROR] Error processing commands: {e}")

# --- INITIALIZATION ---

async def initialize_system(profile_name=None):
    """Initialize both Cortex and ESP32 connections"""
    print("=" * 60)
    print("CORTEX TO ESP32 BRIDGE - INITIALIZING")
    print("=" * 60)
    
    # Initialize Cortex
    if not await connect_to_cortex():
        return False
    
    await asyncio.sleep(1)
    
    if not await authorize_cortex():
        return False
    
    if not await query_headsets():
        return False
    
    if not await control_device():
        return False
    
    if not await create_session():
        return False
    
    # Load profile if specified
    if profile_name:
        await load_profile(profile_name)
    
    if not await subscribe_mental_commands():
        return False
    
    # Initialize ESP32
    if not connect_to_esp32():
        print("[WARNING] ESP32 not connected, will only display commands")
    
    print("\n" + "=" * 60)
    print("SYSTEM READY - Monitoring mental commands...")
    print(f"Trained Command: {TRAINED_COMMAND}")
    print(f"Threshold: {COMMAND_THRESHOLD}")
    print(f"ESP32 Port: {SERIAL_PORT}")
    print(f"Baud Rate: {BAUD_RATE}")
    print("=" * 60 + "\n")
    
    return True

# --- MAIN FUNCTION ---

async def main():
    """Main execution function"""
    
    # Optional: Specify your profile name here
    PROFILE_NAME = None  # Set to your profile name, e.g., "MyProfile"
    
    try:
        if not await initialize_system(PROFILE_NAME):
            print("[ERROR] System initialization failed")
            return
        
        # Start processing commands
        await process_mental_commands()
        
    except KeyboardInterrupt:
        print("\n\nShutting down...")
    except Exception as e:
        print(f"[ERROR] Unexpected error: {e}")
    finally:
        # Cleanup
        if ser and ser.is_open:
            ser.close()
            print("[INFO] ESP32 disconnected")
        if ws:
            await ws.close()
            print("[INFO] Cortex disconnected")
        print("Goodbye!")

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import time
import json
import asyncio
import websockets
import ssl
import serial
import nest_asyncio

nest_asyncio.apply()

# --- CONFIGURATION ---
CORTEX_URL = 'wss://localhost:6868'
CLIENT_ID = "Cxxgmmr3WpMiW61bnzf59flT82ZGEUGgjN8m42lM"
CLIENT_SECRET = "usD2xEZDkWEd35JYZAklttobhFs8Pst7tvDrADR6t4kaTlSCdu2HzNu8agV2VnKmW1VrjwWubpkq2vRVJvDESBGrBE3WbpzncZdEMCSI9BfWl9uQvATCaHL7mxORyQnt"
LICENSE = ""
DEBIT = 1

# Arduino Configuration
SERIAL_PORT = 'COM9'  # Change this to your Arduino port (Windows: COM3, Mac: /dev/tty.usbmodem14201, Linux: /dev/ttyACM0)
BAUD_RATE = 115200

# Command Configuration
TRAINED_COMMAND = 'lift'  # Change to your trained command
COMMAND_THRESHOLD = 0.35   # Sensitivity threshold (0.0 to 1.0)

# --- GLOBAL STATE ---
ws = None
ser = None
auth_token = None
session_id = None
headset_id = None
current_action = ['neutral', 0, 0]

# --- CORTEX API FUNCTIONS ---

async def connect_to_cortex():
    """Establishes WebSocket connection to Emotiv Cortex."""
    global ws
    try:
        ssl_context = ssl.create_default_context()
        ssl_context.check_hostname = False
        ssl_context.verify_mode = ssl.CERT_NONE
        
        ws = await websockets.connect(CORTEX_URL, ssl=ssl_context)
        print("[SUCCESS] Connected to Emotiv Cortex")
        return True
    except Exception as e:
        print(f"[ERROR] Failed to connect to Cortex: {e}")
        return False

async def send_cortex_request(method, params, request_id):
    """Sends a request to Cortex API and returns the response."""
    if not ws:
        print("[ERROR] WebSocket is not connected.")
        return None
    try:
        request = {
            "jsonrpc": "2.0",
            "method": method,
            "params": params,
            "id": request_id
        }
        await ws.send(json.dumps(request))
        response_str = await ws.recv()
        response = json.loads(response_str)
        return response
    except Exception as e:
        print(f"[ERROR] Failed to send/receive request: {e}")
        return None

async def authorize_cortex():
    global auth_token
    params = {
        "clientId": CLIENT_ID,
        "clientSecret": CLIENT_SECRET,
        "license": LICENSE,
        "debit": DEBIT
    }
    response = await send_cortex_request("authorize", params, 4)
    if response and 'result' in response:
        auth_token = response['result']['cortexToken']
        print("[SUCCESS] Authorization successful")
        return True
    print(f"[ERROR] Authorization failed")
    return False

async def query_headsets():
    global headset_id
    response = await send_cortex_request("queryHeadsets", {}, 2)
    if response and 'result' in response and len(response['result']) > 0:
        headset_id = response['result'][0]['id']
        status = response['result'][0].get('status', 'unknown')
        print(f"[SUCCESS] Headset found: {headset_id} (Status: {status})")
        return True
    print("[ERROR] No headset found")
    return False

async def control_device():
    """Connect to the headset"""
    params = {"command": "connect", "headset": headset_id}
    response = await send_cortex_request("controlDevice", params, 3)
    if response and 'result' in response:
        print("[SUCCESS] Device control established")
        return True
    return False

async def create_session():
    global session_id
    params = {
        "cortexToken": auth_token,
        "headset": headset_id,
        "status": "active"
    }
    response = await send_cortex_request("createSession", params, 5)
    
    if response:
        if 'result' in response:
            session_id = response['result']['id']
            print(f"[SUCCESS] Session created: {session_id}")
            return True
        elif 'error' in response:
            error_msg = response['error'].get('message', 'Unknown error')
            print(f"[ERROR] Session creation failed: {error_msg}")
    return False

async def subscribe_mental_commands():
    params = {
        "cortexToken": auth_token,
        "session": session_id,
        "streams": ["com"]
    }
    response = await send_cortex_request("subscribe", params, 6)
    if response and 'result' in response:
        print("[SUCCESS] Subscribed to mental commands")
        return True
    return False

async def load_profile(profile_name):
    """Load a trained profile"""
    params = {
        "cortexToken": auth_token,
        "headset": headset_id,
        "profile": profile_name,
        "status": "load"
    }
    response = await send_cortex_request("setupProfile", params, 7)
    if response and 'result' in response:
        print(f"[SUCCESS] Profile '{profile_name}' loaded")
        return True
    return False

# --- ARDUINO FUNCTIONS ---

def connect_to_arduino():
    """Connects to Arduino via serial port"""
    global ser
    try:
        ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)
        time.sleep(2)  # Wait for Arduino to reset
        print(f"[SUCCESS] Connected to Arduino on {SERIAL_PORT}")
        return True
    except Exception as e:
        print(f"[ERROR] Arduino connection failed: {e}")
        return False

def send_to_arduino(command):
    """Sends command to Arduino"""
    if ser and ser.is_open:
        try:
            ser.write(f"{command}\n".encode('utf-8'))
            print(f"[ARDUINO] Sent: {command}")
            return True
        except Exception as e:
            print(f"[ERROR] Failed to send to Arduino: {e}")
            return False
    return False

# --- MAIN PROCESSING ---

async def process_mental_commands():
    """Processes incoming mental command data and sends to Arduino"""
    global current_action
    
    last_command_sent = None
    
    try:
        while True:
            if not ws:
                break
                
            response = await ws.recv()
            data = json.loads(response)
            
            if 'com' in data:
                action = data['com'][0]
                power = data['com'][1]
                timestamp = data['time']
                
                current_action = [action, power, timestamp]
                
                # Determine command to send based on mental command
                # REVERSED LOGIC: close when NOT detected, open when detected
                if action == TRAINED_COMMAND and power > COMMAND_THRESHOLD:
                    command = "open 20"   # Command when trained action IS active
                else:
                    command = "close 20"  # Command when neutral or command NOT present
                
                # Send command continuously (removed the check to only send on change)
                send_to_arduino(command)
                
                # Only print when command changes to reduce console spam
                if command != last_command_sent:
                    print(f"[COMMAND] Action: {action}, Power: {power:.2f}, Sent: {command}")
                    last_command_sent = command
                
            await asyncio.sleep(0.05)  # Small delay to prevent overwhelming
            
    except Exception as e:
        print(f"[ERROR] Error processing commands: {e}")

# --- INITIALIZATION ---

async def initialize_system(profile_name=None):
    """Initialize both Cortex and Arduino connections"""
    print("=" * 60)
    print("CORTEX TO ARDUINO BRIDGE - INITIALIZING")
    print("=" * 60)
    
    # Initialize Cortex
    if not await connect_to_cortex():
        return False
    
    await asyncio.sleep(1)
    
    if not await authorize_cortex():
        return False
    
    if not await query_headsets():
        return False
    
    if not await control_device():
        return False
    
    if not await create_session():
        return False
    
    # Load profile if specified
    if profile_name:
        await load_profile(profile_name)
    
    if not await subscribe_mental_commands():
        return False
    
    # Initialize Arduino
    if not connect_to_arduino():
        print("[WARNING] Arduino not connected, will only display commands")
    
    print("\n" + "=" * 60)
    print("SYSTEM READY - Monitoring mental commands...")
    print(f"Trained Command: {TRAINED_COMMAND}")
    print(f"Threshold: {COMMAND_THRESHOLD}")
    print("=" * 60 + "\n")
    
    return True

# --- MAIN FUNCTION ---

async def main():
    """Main execution function"""
    
    # Optional: Specify your profile name here
    PROFILE_NAME = None  # Set to your profile name, e.g., "MyProfile"
    
    try:
        if not await initialize_system(PROFILE_NAME):
            print("[ERROR] System initialization failed")
            return
        
        # Start processing commands
        await process_mental_commands()
        
    except KeyboardInterrupt:
        print("\n\nShutting down...")
    except Exception as e:
        print(f"[ERROR] Unexpected error: {e}")
    finally:
        # Cleanup
        if ser and ser.is_open:
            ser.close()
            print("[INFO] Arduino disconnected")
        if ws:
            await ws.close()
            print("[INFO] Cortex disconnected")
        print("Goodbye!")

if __name__ == "__main__":
    asyncio.run(main())